# Liskov Substitution Principle (LSP)

> “Functions that use references to base classes must be able to use objects of derived classes without knowing it.” – Barbara Liskov

## 1. Concept
* If **S** is a subtype of **T**, then objects of type **T** may be replaced with objects of type **S** **without altering program correctness**.
* Sub‑classes must respect **all behavioral contracts** (pre‑conditions ≤, post‑conditions ≥, invariants same) declared by their parent.

## 2. Problematic design that **breaks** LSP

In [ ]:
// BAD: Penguin violates the contract promised by Bird.fly()
class Bird {
  fly(): void {
    console.log("Flapping wings and flying 🐦");
  }
}

class Sparrow extends Bird { }

class Penguin extends Bird {
  fly(): void {
    throw new Error("Penguins cannot fly!");
  }
  
  swim() {
    console.log("Penguin swimming!");
  }
}

function showFlight(bird: Bird) {
  bird.fly();        // caller trusts the Bird contract
}

showFlight(new Sparrow());  // ✅
showFlight(new Penguin());  // ❌ runtime error

Flapping wings and flying 🐦


evalmachine.<anonymous>:15
        throw new Error("Penguins cannot fly!");
        ^

Error: Penguins cannot fly!
    at Penguin.fly (evalmachine.<anonymous>:15:15)
    at showFlight (evalmachine.<anonymous>:20:10)
    at evalmachine.<anonymous>:24:22
    at evalmachine.<anonymous>:26:3
    at sigintHandlersWrap (node:vm:260:12)
    at Script.runInThisContext (node:vm:119:14)
    at Object.runInThisContext (node:vm:297:38)
    at Object.execute (/usr/lib/node_modules/tslab/dist/executor.js:160:38)
    at JupyterHandlerImpl.handleExecuteImpl (/usr/lib/node_modules/tslab/dist/jupyter.js:250:38)
    at /usr/lib/node_modules/tslab/dist/jupyter.js:208:57


### Issues
1. **Contract breach** – `Penguin.fly()` tightens pre‑conditions (requires ability to fly) and fails.
2. Client code now needs `instanceof` checks or `try/catch`, leaking hierarchy details.
3. Base type becomes unreliable; substitutability lost.

## 3. Refactor preserving LSP
Extract behaviour to an interface so only flying birds implement `Flyable`.

In [2]:
interface Flyable {
  fly(): void;
}

class BaseBird { /* common code */ }

class Sparrow extends BaseBird implements Flyable {
  fly() {
    console.log("Sparrow flying!");
  }
}

// Here is the solution: Identify just the common features/behavior
class Penguin extends BaseBird {
  swim() {
    console.log("Penguin swimming!");
  }
}

// Client depends on capability, not full hierarchy
function showFlight(bird: Flyable) {
  bird.fly();
}

showFlight(new Sparrow());     // ✅
// showFlight(new Penguin());  // compile‑time error

Sparrow flying!


### Benefits achieved
* **Valid substitution** – only objects that honour the fly contract can be passed.
* `Penguin` retains its identity without pretending to fly, preventing runtime failures.
* Hierarchy coherence facilitates the Open/Closed Principle.

## 4. Trade‑offs & Constraints
* Potentially **more interfaces** → complexity if over‑segmented (Interface Segregation Principle helps).
* Requires deliberate contract design and documentation.
* Over‑reliance on inheritance can still cause fragile hierarchies; prefer composition when possible.

## 5. Spotting LSP violations – quick checklist
1. Derived class throws exceptions or returns `null` where the base guarantees a value.
2. Sub‑class **requires more** (stronger pre‑conditions) or **delivers less** (weaker post‑conditions).
3. Caller code contains `instanceof`, type casts, or `try/catch` for specific sub‑classes.
4. Swapping parent for child in tests breaks assertions.

## 6. Related GoF patterns
* **Strategy** – favour composition of interchangeable algorithms over inheritance.
* **State** – change behaviour by delegating to state objects rather than subclass overrides.
* **Adapter** – wrap incompatible types instead of inheriting and risking contract violations.

## 7. References
* Barbara Liskov & Jeannette Wing – *A Behavioral Notion of Subtyping* (1994)
* Robert C. Martin – *Design Principles and Design Patterns* (2000)
* https://principles.dev/#lsp